<a href="https://colab.research.google.com/github/julietamponti/tp2Datos-2C2021-/blob/main/3)_Consigna_19_terminado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Cantidad de contenido por planeta fuera de la tierra en la Wikipedia.**

Realizo la instalación de todo lo que utilizare en el desarrollo de la consulta.

In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt update
!apt install openjdk-8-jdk-headless -qq
#!apt install default-jre
#!apt install default-jdk
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 281.3 MB 38 kB/s 
     |████████████████████████████████| 198 kB 63.6 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=6fc9d136c20265e85ad24109a42d689777363077eced5e34869ba4e129947c6c
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hi

Importo librerias.

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd 

Creo el sparkSession.

In [ ]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

Accedo a mi drive para obtener el archivo y luego realizar su lectura.

In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive')
sqlContext = SQLContext(sc)

Mounted at /content/gdrive


/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


Realizo la lectura del archivo geo_tags.csv.

In [ ]:
geo = spark.read.format("csv").option("header", True).load('gdrive/My Drive/TP2 - Datos/geo_tags.csv')
rddGeo = geo.rdd

In [ ]:
rddGeo.take(2)

[Row(gt_id='158041', gt_page_id='4328020', gt_globe='earth', gt_primary='0', gt_lat='41.50749969', gt_lon='-5.81750011', gt_dim='1000', gt_type=None, gt_name=None, gt_country=None, gt_region=None),
 Row(gt_id='5844377', gt_page_id='4670424', gt_globe='earth', gt_primary='0', gt_lat='13.73799992', gt_lon='-89.29199982', gt_dim='10000', gt_type='city', gt_name=None, gt_country=None, gt_region=None)]

Limpio el RDD con el que voy a trabajar, es decir, filtro los None. Y a su vez, realizo el filtrado de los planetas. Ya que por enunciado debo trabajar con contenidos que esten fuera de la tierra.

In [ ]:
rddGeo = rddGeo.filter(lambda x: x.gt_globe != 'earth' and x.gt_globe != None and x.gt_page_id != None)

Realizo un map para quedarme con una estructura del estilo --> ((planeta),1), para luego realizar un reduceByKey y así, obtemer la cantidad de contenidos por planeta.

***Resultado***

In [ ]:
rddGeo.map(lambda x: (x[2],1)).reduceByKey(lambda x,y:x+y).collect()

[('mars', 1438),
 ('titan', 4),
 ('callisto', 3),
 ('titania', 2),
 ('hyperion', 1),
 ('tethys', 1),
 ('mimas', 1),
 ('mercury', 561),
 ('moon', 9061),
 ('ganymede', 116),
 ('venus', 24),
 ('umbriel', 1),
 ('phobos', 20),
 ('oberon', 1),
 ('io', 2),
 ('deimos', 2),
 ('enceladus', 1)]